# Project 2: Supervised Learning
### Building a Student Intervention System

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call

n_students = len(student_data)
n_features = student_data.shape[1] - 1 ## added -1 as last column is the target
n_passed = len(student_data[student_data['passed'] == 'yes'])
n_failed = len(student_data[student_data['passed'] == 'no'])
grad_rate = n_passed / float(n_students) * 100

print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

## Added line of code for investigation - Carl
print X_all.head()  # print the first 5 rows 
print y_all.head()  # print the first 5 rows 

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
   school_GP  school_MS  sex_F  sex_M  age  address_R  address_U  famsize_GT3  \
0          1          0      1      0   18          0          1            1   
1          1          0      1      0   17          0          1            1   
2          1          0      1      0   15          0          1            0   
3          1  

### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset

# Randomly shuffle
X_all, y_all = shuffle(X_all, y_all, random_state=0)

# Divide into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=0)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [13]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Training time (secs): {:.3f}".format(end - start)
    #print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Training time (secs): 0.002
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')


In [14]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Prediction time (secs): {:.3f}".format(end - start)
    #print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Prediction time (secs): 0.000
F1 score for training set: 1.0


In [15]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Prediction time (secs): 0.001
F1 score for test set: 0.712121212121


In [27]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "{}...".format(clf.__class__.__name__)
    print "Training set size: {}".format(len(X_train))
    #print "Test set size: {}".format(len(X_test))  ## added this line to double-check test size
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

# Training size = 300 # full dataset
train_predict(clf, X_train, y_train, X_test, y_test)

# Training size = 200
train_predict(clf, X_train[:200], y_train[:200], X_test, y_test)

# Training size = 100
train_predict(clf, X_train[:100], y_train[:100], X_test, y_test)



------------------------------------------
DecisionTreeClassifier...
Training set size: 300
Training DecisionTreeClassifier...
Training time (secs): 0.005
Predicting labels using DecisionTreeClassifier...
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Prediction time (secs): 0.000
F1 score for test set: 0.723076923077
------------------------------------------
DecisionTreeClassifier...
Training set size: 200
Training DecisionTreeClassifier...
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Prediction time (secs): 0.000
F1 score for test set: 0.644628099174
------------------------------------------
DecisionTreeClassifier...
Training set size: 100
Training DecisionTreeClassifier...
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Prediction time (secs): 0.0

In [29]:
# TODO: Train and predict using two other models
# Nearest Neighbor
from sklearn import neighbors
clfNeighbors = neighbors.KNeighborsClassifier()
# Training size = 300 # full dataset
train_predict(clfNeighbors, X_train, y_train, X_test, y_test)
# Training size = 200
train_predict(clfNeighbors, X_train[:200], y_train[:200], X_test, y_test)
# Training size = 100
train_predict(clfNeighbors, X_train[:100], y_train[:100], X_test, y_test)

print "//////////////////////////////////////////"

from sklearn import svm
clfSVC = svm.SVC()
# Training size = 300 # full dataset
train_predict(clfSVC, X_train, y_train, X_test, y_test)
# Training size = 200
train_predict(clfSVC, X_train[:200], y_train[:200], X_test, y_test)
# Training size = 100
train_predict(clfSVC, X_train[:100], y_train[:100], X_test, y_test)

print "//////////////////////////////////////////"

# Naive Bayes 
from sklearn.naive_bayes import GaussianNB
clfBayes = GaussianNB()
# Training size = 300 # full dataset
train_predict(clfBayes, X_train, y_train, X_test, y_test)
# Training size = 200
train_predict(clfBayes, X_train[:200], y_train[:200], X_test, y_test)
# Training size = 100
train_predict(clfBayes, X_train[:100], y_train[:100], X_test, y_test)

------------------------------------------
KNeighborsClassifier...
Training set size: 300
Training KNeighborsClassifier...
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Prediction time (secs): 0.010
F1 score for training set: 0.845265588915
Predicting labels using KNeighborsClassifier...
Prediction time (secs): 0.004
F1 score for test set: 0.808510638298
------------------------------------------
KNeighborsClassifier...
Training set size: 200
Training KNeighborsClassifier...
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Prediction time (secs): 0.004
F1 score for training set: 0.850174216028
Predicting labels using KNeighborsClassifier...
Prediction time (secs): 0.002
F1 score for test set: 0.771428571429
------------------------------------------
KNeighborsClassifier...
Training set size: 100
Training KNeighborsClassifier...
Training time (secs): 0.000
Predicting labels using KNeighborsClassifier...
Prediction time (secs): 0.0

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [20]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.metrics import make_scorer

f1_custom = make_scorer(f1_score, pos_label='yes')

parameters = {'leaf_size':(5,30),                           
              'n_neighbors':(5,6,7), 
              'weights':('uniform','distance')              
              }

GridSearchNeighbors = GridSearchCV(clfNeighbors, parameters, f1_custom) 
train_predict(GridSearchNeighbors, X_train, y_train, X_test, y_test)

print "\n"
print "GridSearchNeighbors: \n", GridSearchNeighbors

print "\n"
print "Best parameters: \n", GridSearchNeighbors.best_params_


------------------------------------------
GridSearchCV...
Training set size: 300
Training GridSearchCV...
Training time (secs): 0.234
Predicting labels using GridSearchCV...
Prediction time (secs): 0.015
F1 score for training set: 0.825986078886
Predicting labels using GridSearchCV...
Prediction time (secs): 0.005
F1 score for test set: 0.825174825175


GridSearchNeighbors: 
GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=5, p=2, weights='uniform'),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_neighbors': (5, 6, 7), 'weights': ('uniform', 'distance'), 'leaf_size': (5, 30)},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring=make_scorer(f1_score, pos_label=yes), verbose=0)


Best parameters: 
{'n_neighbors': 7, 'weights': 'uniform', 'leaf_size': 5}
